In [1]:
import os # 用于与操作系统交互
import signal # 用于处理信号（如中断信号），可以用来优雅地关闭程序。
import sys # 提供对解释器使用或维护的一些变量的访问，以及与解释器强烈交互的函数。
import time # 提供各种时间相关的功能。
from channel import channel_factory # 用于创建不同类型的通信通道。

In [2]:
from common import const # 可能是定义了一些常量
from config import load_config # 用于加载配置文件。
import random
from plugins import * # 导入所有插件，这通常用于扩展功能。
import threading # 用于多线程编程，可以并发执行任务。
import ast
from config import Config
from config import get_root
from config import get_appdata_dir

In [51]:
from concurrent.futures import Future, ThreadPoolExecutor

In [52]:
handler_pool = ThreadPoolExecutor(max_workers=8)

In [53]:
import  time

In [2]:
import os, platform,sys

In [3]:
platform.system()

'Linux'

In [3]:
try:
    b = u'\u2588' # 定义一个Unicode方块字符
    sys.stdout.write(b + '\r')  # 将方块字符输出到标准输出
    sys.stdout.flush() # 刷新输出
except UnicodeEncodeError:
    BLOCK = 'MM'  # 如果输出失败，使用'MM'作为替代
else:  # 如果成功，使用方块字符
    BLOCK = b

In [49]:
def print_cmd_qr(qrText, white=BLOCK, black='  ', enableCmdQR=True):
    blockCount = int(enableCmdQR)  # 控制是否启用命令行二维码，1表示启用，0表示不启用
    if abs(blockCount) == 0:
        blockCount = 1 # 如果 blockCount 为 0（即二维码未启用），则 blockCount 被重新设置为 1，确保至少绘制一个块（默认行为）。
    white *= abs(blockCount) # 根据 blockCount 调整白色块的大小
    if blockCount < 0:
        white, black = black, white  # 如果 blockCount 为负，交换黑白块颜色
    sys.stdout.write(' '*50 + '\r')  # 清除当前行内容
    sys.stdout.flush() # 刷新输出缓冲区
    qr = qrText.replace('0', white).replace('1', black)  # 根据 qrText 替换 '0' 和 '1' 为相应的块字符
    print(qr)
    sys.stdout.write(qr) # 打印生成的二维码文本
    sys.stdout.flush()  # 刷新输出缓冲区

In [13]:
import requests

In [14]:
s=requests.Session()

In [17]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'

In [20]:
BASE_URL = 'https://login.weixin.qq.com'

In [ ]:
https://login.wx.qq.com/jslogin?appid=wx782c26e4c19acffb&redirect_uri=https%3A%2F%2Flogin.weixin.qq.com%2Fcgi-bin%2Fmmwebwx-bin%2Fwebwxnewloginpage&fun=new&lang=zh_CN&_=1736386032062

In [23]:
encoded_url='https%3A%2F%2Flogin.weixin.qq.com%2Fcgi-bin%2Fmmwebwx-bin%2Fwebwxnewloginpage'

In [24]:
import urllib.parse
decoded_url = urllib.parse.unquote(encoded_url)
print(decoded_url)

https://login.weixin.qq.com/cgi-bin/mmwebwx-bin/webwxnewloginpage


In [27]:
import re

In [34]:
def get_QRuuid():
    url = '%s/jslogin' % BASE_URL # 构造获取二维码 UUID 的请求 URL
    params = {
        'appid': 'wx782c26e4c19acffb',  # 微信登录的应用 ID
        'fun': 'new', # 指定功能为新登录
        'redirect_uri': 'https://login.weixin.qq.com/cgi-bin/mmwebwx-bin/webwxnewloginpage?mod=desktop',  # 重定向 URL
        'lang': 'zh_CN'}  # 指定语言为中文
    headers = {'User-Agent':USER_AGENT} # 设置请求头，指定用户代理
    r = s.get(url, params=params, headers=headers) # 发送 GET 请求获取登录页面数据
    print(r.text)
    # 使用正则表达式提取返回数据中的 'code' 和 'uuid'
    regx = r'window.QRLogin.code = (\d+); window.QRLogin.uuid = "(\S+?)";'
    # 用于在响应文本 r.text 中通过正则表达式 regx 查找特定模式并提取出相应的数据。
    data = re.search(regx, r.text) 
    if data and data.group(1) == '200': # 如果提取成功且返回码为 200
        uuid = data.group(2) # 保存 'uuid' 到实例变量
        return uuid # 返回获取到的 'uuid'

In [43]:
import base64

In [46]:
decoded_bytes = base64.b64decode(uuid)
print(decoded_bytes)

b'\xe1\x9bF\x90\xc51\xc9'


In [56]:
import io
from pyqrcode import QRCode

In [60]:
import subprocess

In [61]:
def print_qr(fileDir):
    subprocess.call(['xdg-open', fileDir])   # Linux：使用 xdg-open 命令打开文件。

In [70]:
def get_QR(uuid=None, enableCmdQR=True, picDir=None, qrCallback=None):
    uuid = uuid 
    picDir = 'QR.png'
    qrStorage = io.BytesIO()
    qrCode = QRCode('https://login.weixin.qq.com/l/' + uuid)
    qrCode.png(qrStorage, scale=10)
    if hasattr(qrCallback, '__call__'):
        qrCallback(uuid=uuid, status='0', qrcode=qrStorage.getvalue())
    else:
        with open(picDir, 'wb') as f:
            f.write(qrStorage.getvalue())
        if enableCmdQR:
           print_cmd_qr(qrCode.text(1), enableCmdQR=enableCmdQR)
        else:
            print_qr(picDir)
    return qrStorage

In [ ]:
ab=get_QR(uuid)

In [47]:
with open('output_file.bin', 'wb') as f:
    f.write(decoded_bytes)
print("文件已保存为 output_file.bin")

文件已保存为 output_file.bin


In [35]:
aa=get_QRuuid()

window.QRLogin.code = 200; window.QRLogin.uuid = "4ZtGkMUxyQ==";


In [44]:
uuid='4ZtGkMUxyQ=='

In [74]:
'%s/cgi-bin/mmwebwx-bin/login' % BASE_URL 

'https://login.weixin.qq.com/cgi-bin/mmwebwx-bin/login'

In [76]:
import time

In [77]:
int(time.time())

1736393002

In [81]:
BASE_URL= 'https://login.weixin.qq.com'

In [86]:
s = requests.Session()

In [102]:
['https://%s/cgi-bin/mmwebwx-bin' % url for url in ('file.wx2.qq.com', 'webpush.wx2.qq.com')]

['https://file.wx2.qq.com/cgi-bin/mmwebwx-bin',
 'https://webpush.wx2.qq.com/cgi-bin/mmwebwx-bin']

In [104]:
import random

In [105]:
'e' + repr(random.random())[2:17]

'e001160098909003'

In [ ]:
for indexUrl, detailedUrl in (
            ("wx2.qq.com", ("file.wx2.qq.com", "webpush.wx2.qq.com")),
            ("wx8.qq.com", ("file.wx8.qq.com", "webpush.wx8.qq.com")),
            ("qq.com", ("file.wx.qq.com", "webpush.wx.qq.com")),
            ("web2.wechat.com", ("file.web2.wechat.com", "webpush.web2.wechat.com")),
            ("wechat.com", ("file.web.wechat.com", "webpush.web.wechat.com"))):
    print(detailedUrl)
    fileUrl, syncUrl = ['https://%s/cgi-bin/mmwebwx-bin' % url for url in detailedUrl]
    print(fileUrl)
    print(syncUrl)

In [95]:
def check_login(uuid=None):
    uuid = uuid
    url = '%s/cgi-bin/mmwebwx-bin/login' % BASE_URL 
    localTime = int(time.time()) # 获取当前的时间戳，用于构建请求参数中的时间部分
    params = 'loginicon=true&uuid=%s&tip=1&r=%s&_=%s' % ( # 构造请求参数字符串，包含 uuid、时间戳、tip 和其他参数
        uuid, int(-localTime / 1579), localTime)
    headers = {'User-Agent': USER_AGENT} # 设置请求头，指定 User-Agent
    r = s.get(url, params=params, headers=headers) # 发送 GET 请求进行登录检查
    regx = r'window.code=(\d+)' # 使用正则表达式从响应中提取返回的 'code' 值
    data = re.search(regx, r.text) # 搜索并提取 'code' 的值
    if data and data.group(1) == '200': # 如果返回码是 200，说明登录成功
        if process_login_info(self, r.text): # 处理登录信息并返回 '200'
            return '200'
        else:
            return '400'  # 登录信息处理失败
    elif data: # 如果返回的 code 不是 200，则返回提取到的 'code' 值
        return data.group(1)
    else: # 如果没有匹配到返回的 code，则返回 400
        return '400'

In [97]:
check_login(uuid)

400


'400'

In [96]:
import uuid
# 生成一个随机 UUID
random_uuid = str(uuid.uuid4())

In [ ]:
import asyncio
import threading
async def async_task(thread_name):
    await asyncio.sleep(1)
    return f"Task completed in {thread_name}"
def thread_function(thread_name):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    print(f"Event loop running in {thread_name}")
    # 调度异步任务
    future = asyncio.run_coroutine_threadsafe(async_task(thread_name), loop)
    result = future.result()  # 阻塞直到任务完成
    print(result)
    loop.close()

# 创建并启动多个线程，每个线程都有自己的事件循环
threads = []
for i in range(5):
    thread_name = f"Thread-{i+1}"
    thread = threading.Thread(target=thread_function, args=(thread_name,))
    threads.append(thread)
    thread.start()
# 等待所有线程完成
for thread in threads:
    thread.join()
print("All threads completed")

In [ ]:
for i,j in os.environ.items():
    print(i,j)

In [3]:
import asyncio
import threading
# 定义一个异步协程
async def my_coroutine():
    print("异步任务执行中")
    await asyncio.sleep(1)
    print("异步任务完成")
# 在子线程中调度协程
def run_in_thread(loop):
    asyncio.run_coroutine_threadsafe(my_coroutine(), loop)
# 主线程中的事件循环
loop = asyncio.get_event_loop()
# 创建并启动一个后台线程
thread = threading.Thread(target=run_in_thread, args=(loop,))
thread.start()
# 继续执行主线程中的任务
print("主线程继续执行")
# 等待子线程完成
thread.join()

主线程继续执行
异步任务执行中
异步任务完成


In [43]:
a={1:2,3:4}

In [45]:
a.clear()

In [37]:
b='' or '默认'

In [47]:
a=[1,2,3]

In [49]:
a=[1]

In [34]:
if not b:
    print(1)

1


In [40]:
b is None

False

In [15]:
a=b or '默认'

In [50]:
a

[1]

In [9]:
a.pop(0)

2

In [2]:
a.append(1)

In [4]:
a.insert(0,3)

In [ ]:
future = asyncio.run_coroutine_threadsafe(my_coroutine(), loop)
# 获取协程结果
result = future.result()  # 阻塞直到协程完成

In [ ]:
# !pip install wechaty

In [9]:
class TrieNode():
    def __init__(self):
        self.Index = 0
        self.Layer = 0 # 节点所在的层级。
        self.End = False # 标记节点是否是敏感词的结尾
        self.Char = '' # 该节点对应的字符。
        # 假设我们有敏感词列表：["ab", "abc", "abcd"]。a 节点是所有三个敏感词的开始（即 "ab", "abc", "abcd"）。
        # 但是，a 不是任何一个敏感词的结尾节点。 因此，a 节点的 Results 列表应该是空的，即 Results = []
        # d 节点是 "abcd" 的结尾节点。同时，d 也是 "ab", "abc", "abcd" 这三个敏感词的结尾节点。
        # d 节点的 Results 列表应该包含所有三个敏感词的索引：[0, 1, 2]，分别表示 "ab"（索引 0）、
        # "abc"（索引 1）、和 "abcd"（索引 2）。
        self.Results = [] # 如果当前节点不是任何敏感词的结尾,results是空列表,如果是,results存储敏感词对应的索引
        self.m_values = {} # m_values 存储当前节点的子节点的字典，键是字符的 ASCII 值，值是子节点。
        self.Failure = None # Failure 指向失配的下一个节点，用于 Aho-Corasick 算法的优化。
        self.Parent = None  # 父节点
    # Add 方法用来添加新的字符节点到当前节点的子节点字典 m_values 中。
    def Add(self,c):
        # 如果该字符节点已经存在，直接返回现有节点
        if c in self.m_values :
            return self.m_values[c] 
        node = TrieNode()  # 创建一个新的子节点
        node.Parent = self  # 设置父节点
        node.Char = c # 设置节点字符
        self.m_values[c] = node # 将子节点添加到 m_values 字典
        return node # 返回新节点
    
    def SetResults(self,index): # index是节点是敏感词结尾，对应的敏感词的索引
        if (self.End == False): # 如果当前节点不是敏感词结尾
            self.End = True  # 将当前节点标记为敏感词结尾
        self.Results.append(index) # 将当前敏感词的索引添加到结果列表

In [51]:
root = TrieNode() # 初始化树根

In [52]:
keywords=['nipples','法轮功','功夫','敏感词','敏感']

In [53]:
allNodeLayer={} # 用于存储所有节点按层级分组
for i in range(len(keywords)): # 遍历所有敏感词
    p = keywords[i] # 获取索引对应的敏感词
    nd = root # 从根节点开始(所有敏感词的根节点)
    for j in range(len(p)):  # 遍历每个敏感词的字符
        nd = nd.Add(ord(p[j])) # 添加字符节点,这时nd从上个父节点指向了新节点(刚刚创建的子节点)
        if (nd.Layer == 0):
            nd.Layer = j + 1 # 设置节点层级
            if nd.Layer in allNodeLayer: # 如果当前层在里面
                allNodeLayer[nd.Layer].append(nd) # 追加
            else: # 否则新建
                allNodeLayer[nd.Layer] = [nd] 
    nd.SetResults(i) 

In [74]:
allNode = [root]  # 包含根节点

In [76]:
allNode[0].Results

[]

In [56]:

        for key in allNodeLayer.keys():  # 遍历所有层级的节点
            allNode.extend(allNodeLayer[key])

[4]

In [4]:
def sigterm_handler_wrap(_signo):
    old_handler = signal.getsignal(_signo) # 获取旧的信号处理器
    # 定义新的信号处理器,参数:第一个是信号编号，第二个是被中断的堆栈帧。
    def func(_signo, _stack_frame):
        # 记录日志
        logger.info("signal {} received, exiting...".format(_signo))
        # 保存用户数据到文件中。
        conf().save_user_datas()
        # 判断old_handler是否可调用
        if callable(old_handler):
            return old_handler(_signo, _stack_frame)
        # 这个在old_handler不可调用时执行,退出程序
        sys.exit(0)
    #将新定义的处理器 func 注册为信号 _signo 的处理器。
    signal.signal(_signo, func) 

In [5]:
from common import singleton
from channel.chat_channel import ChatChannel

In [ ]:
class Channel:
    channel_type = "default"  # 类属性

    def __init__(self, name):
        self.name = name  # 实例属性
# 创建两个实例
channel1 = Channel("Channel 1")
channel2 = Channel("Channel 2")
# 访问类属性
print(Channel.channel_type)  # 输出: default
print(channel1.channel_type)  # 输出: default
print(channel2.channel_type)  # 输出: default
# 修改类属性
Channel.channel_type = "updated"
# 所有实例的类属性都被更新
print(Channel.channel_type)  # 输出: updated
print(channel1.channel_type)  # 输出: updated
print(channel2.channel_type)  # 输出: updated
# 修改实例属性
channel1.name = "New Channel 1"
print(channel1.name)  # 输出: New Channel 1
print(channel2.name)  # 输出: Channel 2

In [11]:
from enum import Enum
class ReplyType(Enum):
    TEXT = 1  # 文本
    VOICE = 2  # 音频文件
    IMAGE = 3  # 图片文件
    IMAGE_URL = 4  # 图片URL
    VIDEO_URL = 5  # 视频URL
    FILE = 6  # 文件
    CARD = 7  # 微信名片，仅支持ntchat
    INVITE_ROOM = 8  # 邀请好友进群
    INFO = 9 # 一般的日志
    ERROR = 10 # 出错的日志
    TEXT_ = 11  # 强制文本
    VIDEO = 12 # 视频文件
    MINIAPP = 13  # 小程序
    def __str__(self):
        return self.name

In [15]:
class Reply:
    def __init__(self, type: ReplyType = None, content=None):
        self.type = type
        self.content = content

    def __str__(self):
        return "Reply(type={}, content={})".format(self.type, self.content)

In [16]:
r=Reply(ReplyType.TEXT,'hello')

In [19]:
class ContextType(Enum): # 消息类型
    TEXT = 1  # 文本消息
    VOICE = 2  # 音频消息
    IMAGE = 3  # 图片消息
    FILE = 4  # 文件信息
    VIDEO = 5  # 视频信息
    SHARING = 6  # 分享信息
    IMAGE_CREATE = 10  # 创建图片命令
    ACCEPT_FRIEND = 19 # 同意好友请求
    JOIN_GROUP = 20  # 加入群聊
    PATPAT = 21  # 拍了拍
    FUNCTION = 22  # 函数调用
    EXIT_GROUP = 23 # 退出群

    def __str__(self):
        return self.name

In [36]:
from queue import Full, Queue # 导入标准库中的 Full 和 Queue 类，Full 是用来表示队列满时的异常
from time import monotonic as time # 从 time 库导入 monotonic 函数，并重命名为 time，用来获取不受系统时钟影响的单调时间

# 定义 Dequeue 类，继承自 Queue 类，添加了 putleft 方法来在队列的左边插入元素
class Dequeue(Queue):
    # putleft 方法用于将元素插入队列的左侧
    def putleft(self, item, block=True, timeout=None):
        # self.not_full 是一个 Condition 对象，通常会与一个锁（Lock 或 RLock）一起使用。进入 with 语句时，Python 会自动获得该锁，从而
        # 确保后续代码块内对共享资源（如队列）的访问是 线程安全 的。程序始终会进入
        # wait() 方法是 Condition 类的一个特有方法， 并不是 self.not_full 特有的，而是所有 Condition 对象共有的方法。它用于使当前线
        # 程 等待某个条件，并在等待期间释放锁。
        with self.not_full:
            if self.maxsize > 0: # 如果队列的最大大小大于0，即有限队列
                if not block: # 如果 block=False，表示不阻塞插入，当队列满时直接抛出 Full 异常
                    if self._qsize() >= self.maxsize:
                        raise Full
                elif timeout is None: # 如果 block=True，并且没有设置 timeout，阻塞等待直到队列有空余空间
                    while self._qsize() >= self.maxsize:
                        self.not_full.wait()
                elif timeout < 0: # 如果 timeout < 0，抛出 ValueError 异常，因为 timeout 必须是非负数
                    raise ValueError("'timeout' must be a non-negative number")
                else: # 如果 timeout 是非负数，计算结束时间，通过阻塞直到队列有空余空间
                    endtime = time() + timeout # 计算结束时间
                    while self._qsize() >= self.maxsize:  # 如果队列已满，阻塞等待直到队列空间空余
                        remaining = endtime - time() # 计算剩余时间
                        if remaining <= 0.0: # 如果剩余时间为负，则超时，抛出 Full 异常
                            raise Full
                        self.not_full.wait(remaining) # 等待剩余时间，直到有空间
            self._putleft(item) # 在队列不满的时候，才会将元素插入队列。
            self.unfinished_tasks += 1 # 增加未完成任务计数器
            self.not_empty.notify() # 唤醒所有等待队列非空的线程，表示队列中有元素
    # putleft_nowait 方法是 putleft 方法的简化版，不进行阻塞，如果队列已满，直接抛出 Full 异常
    def putleft_nowait(self, item):
        return self.putleft(item, block=False)
    # _putleft 方法是将元素放入队列的左边，队列是双端队列（deque）
    def _putleft(self, item):
        self.queue.appendleft(item) 

In [38]:
q=Dequeue()

In [39]:
q.putleft({'1':2})

In [43]:
print(list(q.queue))  # [30, 20, 10]

[{'1': 2}]


In [44]:
type(q.queue)

collections.deque

In [28]:
class Context:
    # 初始化方法
    # type：表示消息的类型，类型应该是 ContextType 枚举中的一种。默认为 None。
    # content：表示消息的内容，可以是任何类型（文本、音频、文件等），默认为 None。
    # kwargs：一个字典，用来存储其他附加的参数，默认为空字典 {}。
    def __init__(self, type: ContextType = None, content=None, kwargs=dict()):
        self.type = type
        self.content = content
        self.kwargs = kwargs
    # 使 Context 对象支持 in 操作符。检查给定的键是否存在于 Context 中，首先检查 type 和 content 属性，然后检查 kwargs 字典。
    # __contains__ 魔法方法可以让 Context 对象支持 in 操作符。因此，通过定义 __contains__ 方法，你可以实现诸如 "origin_ctype"
    # not in context 这样的语法。
    def __contains__(self, key):
        if key == "type":
            return self.type is not None
        elif key == "content":
            return self.content is not None
        else:
            return key in self.kwargs
    # 使 Context 对象支持 [] 操作符来获取属性值。如果键是 type 或 content，返回相应的属性值，否则从 kwargs 字典中获取值。
    def __getitem__(self, key):
        if key == "type":
            return self.type
        elif key == "content":
            return self.content
        else:
            return self.kwargs[key]
    # 提供另一种方式来获取属性的值，并允许设置默认值。如果键存在于 Context 中，返回其值，否则返回默认值。
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    #  使 Context 对象支持 [] 操作符来设置属性值。如果键是 type 或 content，设置相应的属性值，否则将值存储在 kwargs 字典中。
    def __setitem__(self, key, value):
        if key == "type":
            self.type = value
        elif key == "content":
            self.content = value
        else:
            self.kwargs[key] = value
    # 使 Context 对象支持 [] 操作符来删除属性。如果键是 type 或 content，将相应的属性设置为 None，否则从 kwargs 字典中删除该键。
    def __delitem__(self, key):
        if key == "type":
            self.type = None
        elif key == "content":
            self.content = None
        else:
            del self.kwargs[key]
    # 定义 Context 对象的字符串表示形式，方便打印输出
    def __str__(self):
        return "Context(type={}, content={}, kwargs={})".format(self.type, self.content, self.kwargs)

In [29]:
c=Context(ContextType.TEXT,'hello')

In [8]:
def start_channel(channel_name: str):
    # 通过 channel_factory 工厂创建一个相应的通道对象。
    channel = channel_factory.create_channel(channel_name)
    # 检查 channel_name 是否属于预定义的一组特定频道（如微信、飞书、钉钉等）。这个条件的目的是确定是否需要加载插件。
    # wx, wxy: 微信相关,terminal: 终端相关,wechatmp, wechatmp_service: 微信公众号,web: Web应用相关
    # wework: 企业微信,feishu: 飞书,dingtalk: 钉钉
    if channel_name in ["wx", "wxy", "terminal", "wechatmp","web", "wechatmp_service", "wechatcom_app", "wework",
                        const.FEISHU, const.DINGTALK]:
        # 如果 channel_name 在上述特定的频道列表中，那么就通过 PluginManager 类的 load_plugins 方法加载插件。
        PluginManager().load_plugins()
    # 检查配置中是否有一个名为 "use_linkai" 的设置，如果该设置为 True，则会启动一个与 LinkAI 相关的线程。
    # conf() 可能是获取配置的函数，get("use_linkai") 用来获取配置中的 "use_linkai" 值。如果这个值存在且为 
    # True，则执行后续操作。
    if conf().get("use_linkai"):
        # 在 try 块中执行与 LinkAI 相关的操作，尝试导入并启动 LinkAI 客户端。
        # 这里的操作是在 use_linkai 配置项启用时，通过线程启动 LinkAI 客户端。
        try:
            # 导入 common 模块中的 linkai_client 对象，这个对象应该是与 LinkAI 服务交互的客户端。
            from common import linkai_client
            # 作用：通过 threading 模块启动一个新的线程，目标函数是 linkai_client.start，并传入 channel 作为参数。
            # linkai_client.start(channel) 可能是启动 LinkAI 客户端并传入频道对象来进行某种初始化或任务处理。
            # 新线程的启动是为了避免阻塞主程序的执行，确保 LinkAI 客户端能够并行运行。
            # LinkAI 客户端是通过支线线程（子线程）启动的，而主线程则执行 channel.startup()。
            threading.Thread(target=linkai_client.start, args=(channel,)).start()
        # 作用：如果在导入或启动 linkai_client 的过程中发生异常，捕获并忽略这个异常。这里的 pass 语句表示异常发生时
        # 不会执行任何操作，也不会影响程序的其余部分。
        except Exception as e:
            pass
    # 作用：启动 channel，调用 channel 对象的 startup() 方法。这通常是执行频道的初始化任务（如连接、初始化状
    # 态、打开服务等）。
    channel.startup()

In [10]:
aa=signal.getsignal(signal.SIGINT)
callable(aa)

True

In [9]:
def run():
    try:
        # 加载配置和用户数据
        load_config()
        # signal.SIGINT 是一个常量，表示“中断信号”（通常是用户通过按下 Ctrl+C 发送的信号）。
        # 如果没有按下 Ctrl+C，则不会触发 SIGINT 信号，程序将继续按正常流程运行。
        sigterm_handler_wrap(signal.SIGINT)
        # 用户手动终止进程：例如，在命令行使用 kill <pid> 命令，默认情况下发送的就是 SIGTERM。
        sigterm_handler_wrap(signal.SIGTERM)
        # 从配置文件中获取 channel_type 的值，默认为 "wx"
        channel_name = conf().get("channel_type", "wx")
        # sys.argv 包含的是你传递给脚本的命令行参数，以及脚本本身的名称。
        # 如果命令行参数中包含 --cmd，则将 channel_name 设置为 "terminal"，表示使用终端通道。
        if "--cmd" in sys.argv:
            channel_name = "terminal"
        # 根据 channel_name 的值来设置环境变量 WECHATY_LOG，从而控制 Wechaty的日志级别。具体来说，
        if channel_name == "wxy":
            os.environ["WECHATY_LOG"] = "warn" 
        # 调用 start_channel 函数启动指定名称的通道。
        start_channel(channel_name)
        # 使用一个无限循环和 time.sleep(1) 来保持主进程运行，防止程序退出。
        # while True 和 time.sleep(1) 的组合是一种简单的方式，用来确保主进程持续运行，并且可以在接收到终止信号时进行
        # 适当的清理工作。不过，在设计长期运行的应用时，考虑更加优雅的解决方案通常是值得的
        while True:
            time.sleep(1)
    except Exception as e:
        # 捕获所有未处理的异常，并记录错误信息和完整的异常堆栈信息。
        logger.error("App startup failed!")
        logger.exception(e)